# FHIR server commands in Python
- Before proceeding you must start a FHIR server. For default use [hapi-fhir-jpaserver-starter](https://github.com/hapifhir/hapi-fhir-jpaserver-starter). Enter mvn jetty:run from project in cmd to launch server
- [POST file and validate](#POST-file-and-validate)
- [PUT file with ID](#PUT-file-with-ID)
- [GET with validate](#GET-with-validate)
- [GET without validate - Download Resource](#GET-without-validate---Download-Resource)
- [Bulk Export](#Bulk-Export)

In [ ]:
#imports
import requests
import json
import os
from pathlib import Path

In [ ]:
# resource lookup
file_to_resource = {
    'condition': 'Condition',
    'encounter': 'Encounter',
    'encounter_icu': 'Encounter',
    'medadmin_icu': 'MedicationAdministration',
    'medication_administration': 'MedicationAdministration',
    'medication_request': 'MedicationRequest',
    'observation_datetimeevents': 'Observation',
    'observation_labs': 'Observation',
    'observation_micro_org': 'Observation',
    'observation_micro_susc': 'Observation',
    'observation_micro_test': 'Observation',
    'observation_outputevents': 'Observation',
    'organization': 'Organization',
    'patient': 'Patient',
    'procedure': 'Procedure',
    'procedure_icu': 'Procedure'
}

In [ ]:
# read in one resource json
filename = 'patient'
resource = file_to_resource[filename]


fhir_json = Path('/home/alex/Documents/mimic-fhir-test-data/test/' + filename + '.json') 
with open(fhir_json, mode='r') as raw_fhir:
    for line in raw_fhir:
        fhir_data = json.loads(line)
        break

## POST file and validate
- use to validate a resource and get the response right back
- will autogenerate an ID for storing the resource

In [ ]:
# validate resource
fhir_option = '/$validate'
server = 'http://localhost:8080/fhir/'
url = server +  resource + fhir_option

resp = requests.post(url, json = fhir_data, headers={"Content-Type": "application/fhir+json"})
print(resp.text)

## PUT file with ID
- Uploads the resource to the FHIR Server with ID generated in Postgres
- This will check references in resources, so may toss errors if the referenced resources haven't been uploaded yet
- PUT responses:
  - With issue: an OperationOutcome with the detailed issues
  - Without issue: a json representation of the resource

In [ ]:
# validate resource
fhir_option = '/$validate'
server = 'http://localhost:8080/fhir/'
url = server +  resource + '/' + fhir_data['id']

resp = requests.put(url,  json = fhir_data, headers={"Content-Type": "application/fhir+json"} )
print(resp.text)

## GET with validate
- grab resource by ID and validate it
- Resource must have been PUT on the server before this for it to work
- The advantage of using a GET validate versus a POST validate, is that the GET validate will have the text.narrative automatically generated after the PUT call. The narrative is a warning that pops up in the POST.

In [ ]:
url = server +  resource + '/' + fhir_data['id'] + fhir_option
resp = requests.get(url, headers={"Content-Type": "application/fhir+json"})
print(resp.text)

## GET without validate - Download Resource
- Download an individual resource, best to be done after validation complete

In [ ]:
url = server +  resource + '/' + fhir_data['id']
fhir_resource = requests.get(url, headers={"Content-Type": "application/json"})
print(fhir_resource.text)


output_json = Path('/home/alex/Documents/mimic-fhir-test-data/server_output/' + filename + '.json') 
with open(output_json, mode='a+') as output_fhir:
    output_fhir.write(fhir_resource.text.replace('\n','').replace(' ', '') + '\n')

## Bulk Export
- Can export all entries of a particular resource
- May need to enable bulk export for FHIR server: In application.yaml for HAPI FHIR Server need to set parameter `bulk_export_enabled` to true. Restart server to activate this parameter

TO DO:
- Figure out how to poll the bulk export endpoint: http://www.hl7.org/fhir/async.html and https://hl7.org/fhir/uv/bulkdata/export/index.html
- Find the output location
- Output a resource to a file

In [ ]:
url = server +  '$export?_type=' + resource
resp = requests.get(url, headers={"Accept": "json", "Prefer": "respond-async"})
print(resp.text)